# Load rankings and categories

In [6]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json,os,pdb
import tqdm
import warnings

# Choose the universe

Select the number of coins to consider. Right now we are still using a market cap criteria

In [7]:
N = 50

# Load data


In [8]:
df = pd.read_parquet('daily_20221004.parquet.gzip') # modify the path according to your needs
df.columns = [c.split('.')[-1].replace('_','') for c in df.columns]
#df = df.reset_index(level='time_close')
df.index = df.index.set_levels([df.index.levels[0], pd.to_datetime(df.index.levels[1])])

In [9]:
df

Load the listing file and check for the rankings

In [11]:
with open('listing_map_20221004.json',encoding='utf-8') as json_data:
    data = json.load(json_data)
listmap = pd.DataFrame(data['data'])
listmap.sort_values('cmc_rank',inplace=True)
listmap.set_index('cmc_rank',inplace=True)
listmap['symbol_id'] = listmap['symbol']+ '_'+ listmap['id'].astype(str)

In [12]:
listmap

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote,symbol_id
cmc_rank,,,,,,,,,,,,,,,,,
1,1,Bitcoin,BTC,bitcoin,9760,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.916889e+07,1.916889e+07,None,NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 20124.645699098404, 'volume_...",BTC_1
2,1027,Ethereum,ETH,ethereum,6121,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.226484e+08,1.226484e+08,None,NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 1355.3241075801398, 'volume_...",ETH_1027
3,825,Tether,USDT,tether,40468,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,6.794942e+10,7.015545e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 1.0002059770173375, 'volume_...",USDT_825
4,1839,BNB,BNB,bnb,1120,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",2.000000e+08,1.613373e+08,1.613373e+08,None,NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 296.4318964135179, 'volume_2...",BNB_1839
5,3408,USD Coin,USDC,usd-coin,6429,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,4.703592e+10,4.703592e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 1.000153755174831, 'volume_2...",USDC_3408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,13523,Merit Circle,MC,merit-circle,47,2021-10-29T04:07:32.000Z,"[gaming, governance, play-to-earn, token, gami...",1.000000e+09,4.259200e+07,1.000000e+09,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 0.6762041160940436, 'volume_...",MC_13523
497,6727,Reserve,RSV,reserve,16,2020-08-27T00:00:00.000Z,"[defi, stablecoin, asset-backed-stablecoin, co...",NaN,2.885067e+07,2.885067e+07,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 0.9974047085897841, 'volume_...",RSV_6727
498,6739,ONBUFF,ONIT,onbuff,6,2020-08-28T00:00:00.000Z,[ethereum-ecosystem],7.898856e+08,7.700755e+08,7.700755e+08,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 0.037363941965322604, 'volum...",ONIT_6739


In [6]:
df.loc['MPH_7742']

,timeopen,timehigh,timelow,open,high,low,close,volume,marketcap,timestamp,symbol,id
time_close,,,,,,,,,,,,
2020-11-23 00:59:59.999000+00:00,2020-11-23T00:00:00.000Z,2020-11-23T00:00:07.000Z,2020-11-23T00:37:05.000Z,90.554933,90.554933,79.852330,85.599205,1.600851e+07,0.00,2020-11-23T00:59:59.999Z,MPH,7742
2020-11-24 00:59:59.999000+00:00,2020-11-24T00:00:00.000Z,2020-11-24T00:00:07.000Z,2020-11-24T00:40:06.000Z,85.161030,85.161030,80.694133,81.645754,1.009738e+07,0.00,2020-11-24T00:59:59.999Z,MPH,7742
2020-11-25 00:59:59.999000+00:00,2020-11-25T00:00:00.000Z,2020-11-25T00:00:06.000Z,2020-11-25T00:58:05.000Z,57.527366,57.527366,53.679986,53.704963,8.899399e+06,0.00,2020-11-25T00:59:59.999Z,MPH,7742
2020-11-26 00:59:59.999000+00:00,2020-11-26T00:00:00.000Z,2020-11-26T00:56:05.000Z,2020-11-26T00:13:06.000Z,39.466839,40.121814,38.980499,40.056399,4.225065e+06,0.00,2020-11-26T00:59:59.999Z,MPH,7742
2020-11-27 00:59:59.999000+00:00,2020-11-27T00:00:00.000Z,2020-11-27T00:57:05.000Z,2020-11-27T00:16:05.000Z,30.002092,30.391962,29.690305,30.383005,2.933446e+06,0.00,2020-11-27T00:59:59.999Z,MPH,7742
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01 00:59:59.999000+00:00,2022-07-01T00:00:00.000Z,2022-07-01T00:58:00.000Z,2022-07-01T00:27:00.000Z,2.136839,2.161392,2.136423,2.161386,8.780802e+04,1005647.18,2022-07-01T00:59:59.999Z,MPH,7742
2022-07-02 00:59:59.999000+00:00,2022-07-02T00:00:00.000Z,2022-07-02T00:36:00.000Z,2022-07-02T00:07:00.000Z,2.189265,2.210098,2.187692,2.206396,1.075996e+05,1026589.27,2022-07-02T00:59:59.999Z,MPH,7742
2022-07-03 00:59:59.999000+00:00,2022-07-03T00:00:00.000Z,2022-07-03T00:18:00.000Z,2022-07-03T00:58:00.000Z,2.176196,2.177318,2.173795,2.173801,1.020813e+05,1011423.51,2022-07-03T00:59:59.999Z,MPH,7742


# Filter by the universe
Select only the symbols that interest you. According to our previous choice of N, in this case we are filtering for the top 50 by market cap 

Instead of top 50 by market cap, we can also do the ranking for difference between average difference between high and low for each day. 

For cryptos that have high market cap, are there are similarities between their behavior tendency. In other words, will the same LSTM model captures all the behaviors.

To perform this task, we can build one LSTM(fine tune) on one crypto, check how performance differs for different crypto.

Some other things to consider, can we predict close price depends on open price and categorical taking in for symbol (but this has to limit to fewer selected cryptos. 

In [13]:
symbols = list(listmap.loc[:N,'symbol_id'].values)

Remove stablecoins by using the category JSON (available in sakai as the listing file)

In [14]:
with open('category_stablecoin.json',encoding='utf-8') as json_data:
    data = json.load(json_data)
stable_df = pd.json_normalize(data['data']['coins'],max_level=5)
stable_df['symbol_id'] = stable_df['symbol'] + '_' + stable_df['id'].astype(str)
stablecoins = list(stable_df['symbol_id'].values)

In [15]:
# get the top 200 by mkt cap
symbols = set(listmap.loc[:N,'symbol_id'].values) - set(stablecoins)

for l in stablecoins:
    if l in symbols:
        print('Stablecoins in the list!')

In [18]:
symbols

{'AAVE_7278',
 'ADA_2010',
 'ALGO_4030',
 'APE_18876',
 'ATOM_3794',
 'AVAX_5805',
 'AXS_6783',
 'BCH_1831',
 'BNB_1839',
 'BTC_1',
 'CHZ_4066',
 'CRO_3635',
 'DOGE_74',
 'DOT_6636',
 'EGLD_6892',
 'EOS_1765',
 'ETC_1321',
 'ETH_1027',
 'FIL_2280',
 'FLOW_4558',
 'FTT_4195',
 'HBAR_4642',
 'ICP_8916',
 'LEO_3957',
 'LINK_1975',
 'LTC_2',
 'LUNC_4172',
 'MANA_1966',
 'MATIC_3890',
 'NEAR_6535',
 'OKB_3897',
 'QNT_3155',
 'SAND_6210',
 'SHIB_5994',
 'SOL_5426',
 'THETA_2416',
 'TON_11419',
 'TRX_1958',
 'UNI_7083',
 'VET_3077',
 'WBTC_3717',
 'XLM_512',
 'XMR_328',
 'XRP_52',
 'XTZ_2011'}